In [1]:
import pandas as pd
import numpy as np
import datetime

# 1

## Extracción, transformación y carga de los datos de homicidios.xlsx, devolviendo un df 'homicidios' que contiene 716 registros de víctimas fatales asociadas con los 696 hechos a través de un ID.

In [2]:
hechos_hom = pd.read_excel('homicidios.xlsx', sheet_name='HECHOS')
victimas_hom = pd.read_excel('homicidios.xlsx', sheet_name='VICTIMAS')

Se realiza una copia para tener de respaldo en todo momento la visualización del dataset original en caso de precisarla

In [3]:
hechos_h = hechos_hom.copy()
victimas_h = victimas_hom.copy()

Selección de columnas a utilizar. Luego de una visualización del DataFrame, y un entendimiento del problema se decide cuáles son las columnas que aportarán información valiosa para el análisis y se quitan las que no serán utilizadas

In [4]:
hechos_h = hechos_h[['ID','FECHA','AAAA','MM','DD','HH','COMUNA','pos x','pos y','TIPO_DE_CALLE','ACUSADO']]
victimas_h = victimas_h[['ID_hecho','VICTIMA']]

Normalización del nombre de las columnas. Se decide trabajar con la totalidad del nombre de las columnas en mayúsculas, y las separaciones por medio de guion bajo "_".

In [5]:
hechos_h.rename(columns={'AAAA': 'A','MM': 'M', 'DD': 'D', 'HH': 'H',
                         'pos x': 'LONGITUD', 'pos y': 'LATITUD', 'TIPO_DE_CALLE': 'TIPO_CALLE'}, inplace=True)
victimas_h.rename(columns={'ID_hecho': 'ID'}, inplace=True)

In [6]:
hechos_h.shape

(696, 11)

In [7]:
victimas_h.shape

(717, 2)

Merge de ambas tablas, por medio de ID. Se unifican ambas tablas, conservando todos los registros de la tabla víctimas. En algunas (muy pocas) ocasiones, habrá más de un registro que corresponda al mismo hecho, por lo que compartirán ID

In [8]:
homicidios = pd.merge(hechos_h, victimas_h, on='ID', how='right')

In [9]:
homicidios.shape

(717, 12)

Se agrega la columna fatal con valor 1, para luego al unir con el df de lesiones, diferenciar entre accidentes fatales "1" y accidentes con lesiones "0". Se utilizan valores numéricos en lugar de SI/NO, ya que será útil utilizar la columna como un contador de víctimas fatales y realizar operaciones numéricas con ese dato

In [10]:
homicidios['FATAL'] = 1

Se pasa FECHA a formato date

In [11]:
homicidios['FECHA'] = pd.to_datetime(homicidios['FECHA'])

In [12]:
homicidios

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
0,2016-0001,2016-01-01,2016,1,1,4,8,-58.47533969,-34.68757022,AVENIDA,AUTO,MOTO,1
1,2016-0002,2016-01-02,2016,1,2,1,9,-58.50877521,-34.66977709,GRAL PAZ,PASAJEROS,AUTO,1
2,2016-0003,2016-01-03,2016,1,3,7,1,-58.39040293,-34.63189362,AVENIDA,AUTO,MOTO,1
3,2016-0004,2016-01-10,2016,1,10,0,8,-58.46503904,-34.68092974,AVENIDA,SD,MOTO,1
4,2016-0005,2016-01-21,2016,1,21,5,1,-58.38718297,-34.62246630,AVENIDA,PASAJEROS,MOTO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,6,3,-58.40596860,-34.61011987,AVENIDA,AUTO,PEATON,1
713,2021-0093,2021-12-13,2021,12,13,17,7,-58.43353773,-34.64561636,AVENIDA,AUTO,MOTO,1
714,2021-0094,2021-12-20,2021,12,20,1,9,-58.46739825,-34.65117757,AUTOPISTA,AUTO,MOTO,1
715,2021-0095,2021-12-30,2021,12,30,0,11,-58.47293407,-34.61984745,AVENIDA,CARGAS,MOTO,1


Se elimina el único registro sin dato de Hora ("H")

In [13]:
homicidios[homicidios['H'] == 'SD']

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
537,2019-0103,2019-12-18,2019,12,18,SD,11,-58.52169422,-34.59471640,GRAL PAZ,MOTO,MOTO,1


In [14]:
homicidios = homicidios[homicidios['H'] != 'SD']

Se pasa la columna de Hora "H" a entero (int)

In [15]:
homicidios = homicidios.copy()
homicidios['H'] = homicidios['H'].astype(int)

Se corrobora si hay registros nulos

In [16]:
homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 716 entries, 0 to 716
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          716 non-null    object        
 1   FECHA       716 non-null    datetime64[ns]
 2   A           716 non-null    int64         
 3   M           716 non-null    int64         
 4   D           716 non-null    int64         
 5   H           716 non-null    int32         
 6   COMUNA      716 non-null    int64         
 7   LONGITUD    716 non-null    object        
 8   LATITUD     716 non-null    object        
 9   TIPO_CALLE  716 non-null    object        
 10  ACUSADO     716 non-null    object        
 11  VICTIMA     716 non-null    object        
 12  FATAL       716 non-null    int64         
dtypes: datetime64[ns](1), int32(1), int64(5), object(6)
memory usage: 75.5+ KB


***

# 2

## Extracción, transformación y carga de los datos de lesiones.xlsx, devolviendo un df 'lesiones' que contiene 27590 registros de víctimas con lesiones asociadas con los 23785 hechos a través de un ID.

In [17]:
hechos_les = pd.read_excel('lesiones.xlsx', sheet_name='HECHOS')
victimas_les = pd.read_excel('lesiones.xlsx', sheet_name='VICTIMAS')

Se realiza una copia para tener de respaldo en todo momento la visualización del dataset original en caso de precisarla

In [18]:
hechos_l = hechos_les.copy()
victimas_l = victimas_les.copy()

Selección de columnas a utilizar. Luego de una visualización del DataFrame, y un entendimiento del problema se decide cuáles son las columnas que aportarán información valiosa para el análisis y se quitan las que no serán utilizadas

In [19]:
hechos_l = hechos_l[['id','fecha','aaaa','mm','dd','franja_hora','comuna','longitud','latutid','tipo_calle','acusado']]
victimas_l = victimas_l[['ID hecho','VEHICULO_VICTIMA']]

Normalización del nombre de las columnas. Se decide trabajar con la totalidad del nombre de las columnas en mayúsculas, y las separaciones por medio de guion bajo "_".

In [20]:
hechos_l.rename(columns={'id':'ID','fecha':'FECHA','aaaa':'A','mm':'M','dd':'D','franja_hora':'H',
                         'comuna':'COMUNA','longitud':'LONGITUD','latutid':'LATITUD','tipo_calle':'TIPO_CALLE','acusado':'ACUSADO'}, inplace=True)
victimas_l.rename(columns={'ID hecho':'ID','VEHICULO_VICTIMA':'VICTIMA'}, inplace=True)


In [21]:
hechos_l.shape

(23785, 11)

In [22]:
victimas_l.shape

(27605, 2)

Merge de ambas tablas, por medio de ID. Se unifican ambas tablas, conservando todos los registros de la tabla víctimas. En algunas (mmuy pocas) ocasiones, habrá más de un registro que corresponda al mismo hecho, por lo que compartirán ID

In [23]:
lesiones = pd.merge(hechos_l, victimas_l, on='ID', how='right')

In [24]:
lesiones.shape

(27605, 12)

Se agrega la columna fatal con valor 0, para luego al unir con el df de lesiones, diferenciar entre accidentes fatales "1" y accidentes con lesiones "0". Se utilizan valores numéricos en lugar de SI/NO, ya que será útil utilizar la columna como un contador de víctimas fatales y realizar operaciones numéricas con ese dato

In [25]:
lesiones['FATAL'] = 0

Se pasa FECHA a formato date

In [26]:
lesiones['FECHA'] = pd.to_datetime(lesiones['FECHA'])

In [27]:
lesiones

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
0,LC-2019-0000053,2019-01-01,2019.0,1.0,1.0,1,8,-58.44351,-34.669125,SD,SD,sd,0
1,LC-2019-0000063,2019-01-01,2019.0,1.0,1.0,2,8,-58.468335,-34.677556,SD,SD,sd,0
2,LC-2019-0000079,2019-01-01,2019.0,1.0,1.0,2,7,-58.437425,-34.647349,SD,SD,sd,0
3,LC-2019-0000082,2019-01-01,2019.0,1.0,1.0,4,3,-58.398225,-34.604579,SD,SD,sd,0
4,LC-2019-0000082,2019-01-01,2019.0,1.0,1.0,4,3,-58.398225,-34.604579,SD,SD,sd,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021-09-11,2021.0,9.0,11.0,18,14,-58.420119387377,-34.581370448309,AVENIDA,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
27601,LC-2021-0530228,2021-10-25,2021.0,10.0,25.0,12,14,-58.406897,-34.581142,SD,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
27602,LC-2021-0530228,2021-10-25,2021.0,10.0,25.0,12,14,-58.406897,-34.581142,SD,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
27603,LC-2021-0201378,2021-05-02,2021.0,5.0,2.0,0,SD,SD,SD,SD,OBJETO FIJO,MOTO,0


### Tratamiento de valores nulos

Se corrobora la existencia de registros con valores nulos

In [28]:
lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          27605 non-null  object        
 1   FECHA       27598 non-null  datetime64[ns]
 2   A           27598 non-null  float64       
 3   M           27598 non-null  float64       
 4   D           27598 non-null  float64       
 5   H           27593 non-null  object        
 6   COMUNA      27420 non-null  object        
 7   LONGITUD    27328 non-null  object        
 8   LATITUD     27328 non-null  object        
 9   TIPO_CALLE  27598 non-null  object        
 10  ACUSADO     27598 non-null  object        
 11  VICTIMA     27605 non-null  object        
 12  FATAL       27605 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(8)
memory usage: 2.7+ MB


Se borran los 7 valores con Año ("A") nulo, ya que son registros que no tienen datos en la gran mayoría de sus columnas

In [29]:
lesiones[lesiones['A'].isna()]

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
11790,PNA-2019-0005246,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sd,0
16186,HC-2020-0499647,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AUTO,0
16187,HC-2020-0499647,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AUTO,0
16188,HC-2020-0499647,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AUTO,0
22821,HC-2021-0147160,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SD,0
22822,HC-2021-0147160,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SD,0
26799,HC-2021-0167039,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEATON,0


In [30]:
lesiones.dropna(subset=['A'], inplace=True)

Se pasan las columnas Año, Mes y Día a tipo de dato entero (int)

In [31]:
lesiones = lesiones.copy()
lesiones['A'] = lesiones['A'].astype(int)

In [32]:
lesiones = lesiones.copy()
lesiones['M'] = lesiones['M'].astype(int)

In [33]:
lesiones = lesiones.copy()
lesiones['D'] = lesiones['D'].astype(int)

Se eliminan los tres registros sin dato de Hora ("H")

In [34]:
lesiones[lesiones['H'] == 'sd']

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
11783,PFA-2019-0000120,2019-02-26,2019,2,26,sd,NaN,NaN,NaN,SD,SD,sd,0
11784,PFA-2019-0000301,2019-11-04,2019,11,4,sd,NaN,NaN,NaN,SD,SD,sd,0
18744,LC-2021-0448511,2021-08-23,2021,8,23,sd,SD,SD,SD,SD,SD,SD,0


In [35]:
lesiones = lesiones[lesiones['H'] != 'sd']
lesiones = lesiones.dropna(subset=['H'])

Se pasa Hora "H" a tipo de dato entero (int)

In [36]:
lesiones = lesiones.copy()
lesiones['H'] = lesiones['H'].astype(int)

Corroboración de la medida del df

In [38]:
lesiones.shape

(27590, 13)

***

### Es opcional la exportación de respaldo del df homicidios y el df lesiones con los merge y normalizaciones ya realizados, antes de concatenarlos.

In [248]:
#homicidios.to_csv('homicidiosETL.csv', index=False)
#lesiones.to_csv('lesionesETL.csv', index=False)
#hom = pd.read_csv('homicidiosETL.csv')
#les = pd.read_csv('lesionesETL.csv')

***

# 3

## Concatenación de ambos df (homicidios y lesiones), en un archivo total, llamado siniestrosETL.csv, donde se encuentran los 28306 registros totales que incluyen las 28306 víctimas (716 fatales + 27590 lesionadas) y se relacionan con un hecho específico de los 24481 hechos registrados (23785 con lesiones + 696 con víctimas fatales)

Se realiza una copia para tener de respaldo en todo momento la visualización de los dataset transformados, antes de su concatenación, en caso de precisarla

In [39]:
hom = homicidios.copy()
les = lesiones.copy()

Concatenación de ambos df (lesiones y homiciios) en un nuevo DataFrame llamado "siniestros"

In [40]:
siniestros = pd.concat([hom, les], ignore_index=True)

In [41]:
siniestros

,ID,FECHA,A,M,D,H,COMUNA,LONGITUD,LATITUD,TIPO_CALLE,ACUSADO,VICTIMA,FATAL
0,2016-0001,2016-01-01,2016,1,1,4,8,-58.47533969,-34.68757022,AVENIDA,AUTO,MOTO,1
1,2016-0002,2016-01-02,2016,1,2,1,9,-58.50877521,-34.66977709,GRAL PAZ,PASAJEROS,AUTO,1
2,2016-0003,2016-01-03,2016,1,3,7,1,-58.39040293,-34.63189362,AVENIDA,AUTO,MOTO,1
3,2016-0004,2016-01-10,2016,1,10,0,8,-58.46503904,-34.68092974,AVENIDA,SD,MOTO,1
4,2016-0005,2016-01-21,2016,1,21,5,1,-58.38718297,-34.62246630,AVENIDA,PASAJEROS,MOTO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28301,LC-2021-0451911,2021-09-11,2021,9,11,18,14,-58.420119387377,-34.581370448309,AVENIDA,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
28302,LC-2021-0530228,2021-10-25,2021,10,25,12,14,-58.406897,-34.581142,SD,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
28303,LC-2021-0530228,2021-10-25,2021,10,25,12,14,-58.406897,-34.581142,SD,TRANSPORTE PUBLICO,TRANSPORTE PUBLICO,0
28304,LC-2021-0201378,2021-05-02,2021,5,2,0,SD,SD,SD,SD,OBJETO FIJO,MOTO,0


Comprobación de columnas con valores nulos

In [42]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28306 entries, 0 to 28305
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          28306 non-null  object        
 1   FECHA       28306 non-null  datetime64[ns]
 2   A           28306 non-null  int64         
 3   M           28306 non-null  int64         
 4   D           28306 non-null  int64         
 5   H           28306 non-null  int32         
 6   COMUNA      28130 non-null  object        
 7   LONGITUD    28038 non-null  object        
 8   LATITUD     28038 non-null  object        
 9   TIPO_CALLE  28306 non-null  object        
 10  ACUSADO     28306 non-null  object        
 11  VICTIMA     28306 non-null  object        
 12  FATAL       28306 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(4), object(7)
memory usage: 2.7+ MB


Exportación del archivo siniestrosETL.csv

In [43]:
siniestros.to_csv('siniestrosETL.csv', index=False)

***